In [7]:
# Cell 1 - Import required libraries
import numpy as np
import pandas as pd
import glob
import os
import tensorflow as tf
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.saving import register_keras_serializable
import joblib  # For loading the saved scaler

In [8]:
# Cell 2 - Define Transformer model classes (needed for loading the saved model)
@register_keras_serializable()
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        if embed_dim % num_heads != 0:
            raise ValueError(f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}")
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.key_dense = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.value_dense = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.combine_heads = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.dropout = layers.Dropout(dropout_rate)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dropout_rate": self.dropout_rate,
        })
        return config

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        weights = self.dropout(weights)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs, training=False):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

@register_keras_serializable()
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.2, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = MultiHeadSelfAttention(embed_dim, num_heads, rate)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu", kernel_regularizer=l2(0.01)),
            layers.Dropout(rate),
            layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate,
        })
        return config

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

@register_keras_serializable()
class TransformerClassifier(Model):
    def __init__(self, num_classes, embed_dim, num_heads, ff_dim, num_layers, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_layers = num_layers
        
        self.input_norm = layers.BatchNormalization()
        self.dense_input = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.input_dropout = layers.Dropout(0.2)
        
        self.transformer_blocks = [
            TransformerBlock(embed_dim, num_heads, ff_dim) for _ in range(num_layers)
        ]
        
        self.global_average_pooling = layers.GlobalAveragePooling1D()
        self.dropout1 = layers.Dropout(0.2)
        self.dense1 = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01))
        self.dropout2 = layers.Dropout(0.2)
        self.dense2 = layers.Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": self.num_classes,
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "num_layers": self.num_layers,
        })
        return config

    def call(self, inputs, training=False):
        x = self.input_norm(inputs)
        x = self.dense_input(x)
        x = self.input_dropout(x, training=training)
        
        for transformer in self.transformer_blocks:
            x = transformer(x, training=training)
            
        x = self.global_average_pooling(x)
        x = self.dropout1(x, training=training)
        x = self.dense1(x)
        x = self.dropout2(x, training=training)
        return self.dense2(x)

In [9]:
# Cell 3 - Define feature extraction functions
def extract_time_domain_features(data):
    """Extract time domain features from the input data"""
    means = np.mean(data, axis=1)
    stds = np.std(data, axis=1)
    mins = np.min(data, axis=1)
    maxs = np.max(data, axis=1)
    percentile_25 = np.percentile(data, 25, axis=1)
    percentile_50 = np.percentile(data, 50, axis=1)
    percentile_75 = np.percentile(data, 75, axis=1)
    
    time_features = np.column_stack([
        means, stds, mins, maxs,
        percentile_25, percentile_50, percentile_75
    ])
    
    return time_features

def extract_frequency_domain_features(data):
    """Extract frequency domain features from the input data"""
    freq_features = np.fft.fft(data, axis=1)
    freq_magnitude = np.abs(freq_features)
    
    dominant_freqs = np.argmax(freq_magnitude, axis=1)
    freq_energies = np.sum(freq_magnitude, axis=1)
    
    freq_features = np.column_stack([dominant_freqs, freq_energies])
    
    return freq_features

def extract_statistical_features(data):
    """Extract statistical features from the input data"""
    means = np.mean(data, axis=1)
    medians = np.median(data, axis=1)
    variances = np.var(data, axis=1)
    skewness = skew(data, axis=1)
    kurtosis_vals = kurtosis(data, axis=1)
    
    statistical_features = np.column_stack([
        means, medians, variances, skewness, kurtosis_vals
    ])
    
    return statistical_features

In [15]:
# Cell 4 - Define preprocessing and classification functions
def load_and_preprocess_file(file_path, scaler):
    """
    Load and preprocess a single file for classification
    """
    try:
        # Read the file
        with open(file_path, 'r') as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]

        # Convert to DataFrame
        df = pd.DataFrame([line.split() for line in lines])
        
        if df.empty:
            raise ValueError(f"File is empty after processing")

        # First row contains total number of rows
        total_rows = int(df.iloc[0, 0])
        
        # Extract feature data
        data = df.iloc[1:, :].values
        
        # Ensure each row has 7 features
        reshaped_data = []
        for row in data:
            if len(row) == 7:
                reshaped_data.append(row)
        
        if not reshaped_data:
            raise ValueError("No valid data rows found")
        
        # Convert to numpy array
        reshaped_data = np.array(reshaped_data, dtype=float)
        
        # Extract features
        time_features = extract_time_domain_features(reshaped_data)
        freq_features = extract_frequency_domain_features(reshaped_data)
        stat_features = extract_statistical_features(reshaped_data)
        
        # Combine features
        combined_features = np.concatenate((time_features, freq_features, stat_features), axis=1)
        combined_features = scaler.transform(combined_features)
        return combined_features
        
    except Exception as e:
        raise Exception(f"Error processing file {file_path}: {e}")

def classify_emotion(model_path, scaler_path, file_path):
    """
    Classify emotion using saved model and scaler and return confidences for all emotions
    """
    try:
        # Load the saved model and scaler
        model = load_model(model_path)
        scaler = joblib.load(scaler_path)
        
        # Process the input file
        features = load_and_preprocess_file(file_path, scaler)
        
        # Make prediction
        prediction = model.predict(features)
        
        # Calculate mean probabilities across all sequences
        mean_probabilities = np.mean(prediction, axis=0)
        
        # Map class indices to labels
        class_labels = ['Normal', 'Depression', 'Anxiety', 'Stress']
        
        # Create sorted list of (emotion, confidence) tuples
        emotion_confidences = [
            (label, float(prob * 100))
            for label, prob in zip(class_labels, mean_probabilities)
        ]
        # Sort by confidence in descending order
        emotion_confidences.sort(key=lambda x: x[1], reverse=True)
        
        # The predicted emotion is the first one (highest confidence)
        predicted_emotion = emotion_confidences[0][0]
        
        return predicted_emotion, emotion_confidences
        
    except Exception as e:
        raise Exception(f"Classification error: {e}")

def classify_directory(model_path, scaler_path, directory_path):
    """
    Classify all .svc files in a directory
    """
    results = []
    
    # Find all .svc files
    file_paths = glob.glob(os.path.join(directory_path, "*.svc"))
    
    if not file_paths:
        raise ValueError("No .svc files found in the directory")
    
    for file_path in file_paths:
        try:
            predicted_emotion, emotion_confidences = classify_emotion(
                model_path, scaler_path, file_path
            )
            results.append({
                'file': os.path.basename(file_path),
                'predicted_emotion': predicted_emotion,
                'confidences': emotion_confidences
            })
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue
    
    return results


In [16]:
# Cell 5 - Run the classifier
# Set the paths
model_path = './saved_models/emotion_classifier.keras'
scaler_path = './saved_models/scaler.save'
input_directory = './input_files/'

try:
    # Example 1: Classify a single file
    file_path = './input_files/0_hw00001.svc'
    predicted_emotion, confidences = classify_emotion(model_path, scaler_path, file_path)
    print(f"\nFile: {os.path.basename(file_path)}")
    print(f"Predicted Emotion: {predicted_emotion}")
    print("\nConfidence Levels (Highest to Lowest):")
    for emotion, confidence in confidences:
        print(f"{emotion}: {confidence:.2f}%")
    
    # Example 2: Classify all files in a directory
    results = classify_directory(model_path, scaler_path, input_directory)
    
    print("\nDirectory Classification Results:")
    print("================================")
    for result in results:
        print(f"\nFile: {result['file']}")
        print(f"Predicted Emotion: {result['predicted_emotion']}")
        print("\nConfidence Levels (Highest to Lowest):")
        for emotion, confidence in result['confidences']:
            print(f"{emotion}: {confidence:.2f}%")
            
except Exception as e:
    print(f"Classification error: {e}")

83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step

File: 0_hw00001.svc
Predicted Emotion: Normal

Confidence Levels (Highest to Lowest):
Normal: 99.74%
Stress: 0.20%
Depression: 0.05%
Anxiety: 0.00%
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
97/97 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
177/177 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step

Directory Classification Results:

File: 0_hw00001.svc
Predicted Emotion: Normal

Confidence Levels (Highest to Lowest):
Normal: 99.74%
Stress: 0.20%
Depression: 0.05%
Anxiety: 0.00%

File: 1_hw00001.svc
Predicted Emotion: Depression

Confidence Levels (Highest to Lowest):
Depression: 99.46%
Normal: 0.49%
Anxiety: 0.04%
Stress: 0.00%

File: 2_hw00003.svc
Predicted Emotion: Anxiety

Confidence Levels (Highest to Lowest):
Anxiety: 99.82%
Stress: 0.15%
Depression: 0.02%
Normal: 0.01%

File: 3_synthetichw00006.svc
Predicted 